In [1]:
%pip install mediapipe opencv-python

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import mediapipe as mp 
import numpy as np 
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
help(mp_drawing.DrawingSpec)

Help on class DrawingSpec in module mediapipe.python.solutions.drawing_utils:

class DrawingSpec(builtins.object)
 |  DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2) -> None
 |  
 |  DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2)
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __init__(self, color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  -------------------------------------------------

In [4]:
from IPython.display import Image

Image(url="https://i.imgur.com/3j8BPdc.png", width=300)

In [5]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[x: 0.5209637
y: 0.5704421
z: -1.0694225
visibility: 0.9996762
, x: 0.5476559
y: 0.5249866
z: -1.0334064
visibility: 0.9987294
, x: 0.5660116
y: 0.52555716
z: -1.0336852
visibility: 0.999151
, x: 0.58191633
y: 0.5271524
z: -1.033791
visibility: 0.99863064
, x: 0.501453
y: 0.5260663
z: -1.0134323
visibility: 0.9985494
, x: 0.48918763
y: 0.52740586
z: -1.013443
visibility: 0.9988921
, x: 0.47775394
y: 0.52909
z: -1.0140357
visibility: 0.99848086
, x: 0.61002386
y: 0.56067795
z: -0.7786441
visibility: 0.9991823
, x: 0.46583405
y: 0.5619612
z: -0.66545063
visibility: 0.9993693
, x: 0.5584211
y: 0.637117
z: -0.9735834
visibility: 0.99967635
, x: 0.5008386
y: 0.63176554
z: -0.9423178
visibility: 0.99969435
, x: 0.7528081
y: 0.88822645
z: -0.5900021
visibility: 0.99443495
, x: 0.3450641
y: 0.8544743
z: -0.47370806
visibility: 0.9951178
, x: 0.8691968
y: 1.1889817
z: -0.7170566
visibility: 0.22248785
, x: 0.14994365
y: 1.0457146
z: -0.45394984
visibility: 0.7995637
, x: 0.8636459
y: 1.4453562


In [6]:
len(landmarks)

33

In [7]:
for lndmrk in mp_pose.PoseLandmark: 
    print(lndmrk)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [8]:
[landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

[0.32037389278411865, 0.8734630942344666]

In [9]:
[landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]

[0.15229347348213196, 1.0651872158050537]

In [10]:
[landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

[0.10432340949773788, 1.4515455961227417]

In [11]:
def calculate_overhead_press_angle(right_shoulder, right_elbow, right_wrist):
    right_shoulder = np.array(right_shoulder)  # Shoulder landmark
    right_elbow = np.array(right_elbow)  # Elbow landmark
    right_wrist = np.array(right_wrist)  # Wrist landmark
    
    # Calculate the angle between the vectors formed by the landmarks
    angle = np.degrees(np.arccos(np.dot(right_shoulder - right_elbow, right_wrist - right_elbow) / (np.linalg.norm(right_shoulder - right_elbow) * np.linalg.norm(right_wrist - right_elbow))))
    
    return angle


In [12]:
right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]


In [13]:
right_shoulder, right_elbow, right_wrist

([0.32037389278411865, 0.8734630942344666],
 [0.15229347348213196, 1.0651872158050537],
 [0.10432340949773788, 1.4515455961227417])

In [14]:
calculate_overhead_press_angle(right_shoulder, right_elbow, right_wrist)

145.83724626223358

In [15]:
tuple(np.multiply(right_shoulder, [640, 480]).astype(int))
tuple(np.multiply(right_elbow, [640, 480]).astype(int))
tuple(np.multiply(right_wrist, [640, 480]).astype(int))

(66, 696)

Start of form checking


In [19]:
landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]

x: 0.15229347
y: 1.0651872
z: -0.29270393
visibility: 0.58048993

In [20]:
landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

x: 0.3203739
y: 0.8734631
z: -0.32531524
visibility: 0.9992571

In [21]:
landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

x: 0.37531424
y: 1.5613111
z: 0.034757502
visibility: 0.0004910473

In [22]:
def calculate_form_angle(elbow, shoulder, hip):
    elbow = np.array(elbow)  # Right elbow landmark
    shoulder = np.array(shoulder)  # Right shoulder landmark
    hip = np.array(hip)      # Right hip landmark
    
    
    radians = np.arctan2(shoulder[1]-hip[1], shoulder[0]-hip[0]) - np.arctan2(elbow[1]-hip[1], elbow[0]-hip[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle


In [23]:
elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]


In [24]:
calculate_form_angle(elbow, shoulder, hip)

19.638493359787127

In [25]:
tuple(np.multiply(right_shoulder, [640, 480]).astype(int))
tuple(np.multiply(right_elbow, [640, 480]).astype(int))
tuple(np.multiply(right_wrist, [640, 480]).astype(int))

(66, 696)

In [26]:
cap = cv2.VideoCapture(0)

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for ohp landmarks
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            # Calculate ohp angle
            overhead_angle = calculate_overhead_press_angle(right_shoulder, right_elbow, right_wrist)
            form_angle = calculate_form_angle(elbow, shoulder, hip)

            
            # Visualize angle
            cv2.putText(image, str(overhead_angle), 
                           tuple(np.multiply(right_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            cv2.putText(image, str(form_angle), 
                           tuple(np.multiply(shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Overhead Press Checker', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [27]:
cap = cv2.VideoCapture(0)

# OHP counter variables
counter = 0 
stage = None
feedback = ""

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for ohp landmarks
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            
            
            # Calculate ohp angle
            overhead_angle = calculate_overhead_press_angle(right_shoulder, right_elbow, right_wrist)
            form_angle = calculate_form_angle(elbow, shoulder, hip)
            
            # Visualize angle
            cv2.putText(image, str(overhead_angle), 
                           tuple(np.multiply(right_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            cv2.putText(image, str(form_angle), 
                           tuple(np.multiply(shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Overhead Press Counter Algorithm
            if overhead_angle < 90:
                stage = "down" 
                if form_angle < 21:
                    feedback = "Tuck elbows in"
                elif form_angle > 21:
                    feedback ="Good form"
            if overhead_angle > 150 and stage =="down":
                stage="up"
                if form_angle < 21:
                    feedback = "DW"
                counter += 1
            
                       
        except:
            pass
        
        # Render Overhead Press Counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
         # Text for 'SQUATS'
        cv2.putText(image, 'BENCH PRESS', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2, cv2.LINE_AA)

        # Text for 'STAGE'
        cv2.putText(image, 'STAGE', (165, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, (160, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2, cv2.LINE_AA)

        # Text for 'FORM' (Ensure it fits within the box)
        feedback_lines = feedback.split('\n')
        if len(feedback_lines) > 0:
            feedback_line1 = feedback_lines[0]
        else:
            feedback_line1 = ''

        if len(feedback_lines) > 1:
            feedback_line2 = feedback_lines[1]
        else:
            feedback_line2 = ''

        cv2.putText(image, 'FORM', (290, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, feedback_line1, (270, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, feedback_line2, (270, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Bench Press Tracker', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    
    cv2.destroyAllWindows()
